<a href="https://colab.research.google.com/github/russodanielp/intro_cheminformatics/blob/google_colab/Lab%2008%20-%20Machine%20Learning%20-%20Supervised%2C%20QSAR/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QSAR

## Aim of this lab

To use different machine learning alogorithms for Quantitative Structure Activity Relationship Modeling. 

### Objectives

* Use a variety of machine learning algorithms to develop QSAR models


## Background


Quantitative Structure-Activity Relationahsip (QSAR) modeling is one of the most common and useful methodologies in cheminformatics.  QSAR seeks to use machine learning algorithms to map chemical information to a biological activity.  The models can then be used for predictive purposes.

## Data:

__You'll have to do similar procedures for data__

Took from PubChem AID [743079](https://pubchem.ncbi.nlm.nih.gov/bioassay/743079#section=Result-Definitions)

Go to download data table.

You'll need to find the end points of interest (Activity Outcome, AC50, etc.).  They vary by assay.  

### Classification:

-Used PubChem `PUBCHEM_ACTIVITY_OUTCOME`. 
-Removed Records with no SMILES/CID.
-Removed Records with Inconclusive.  
-Balance Active Responses with Inactive response (more on this next lecture).  
-Made a new columns with a 1 if the chemical was Active 0 if it was Inactive 

### Regression:
-Used Column `Fit_LogAC50-Replicate_2` (you would ideally do all replicates and average results).   
-Only Used `Active` Chemicals.   

In [ ]:
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import Descriptors


def calc_fp_from_mol(mol, method="maccs", n_bits=2048):
    """
    Encode a molecule from a RDKit Mol into a fingerprint.

    Parameters
    ----------
    mol : RDKit Mol
        The RDKit molecule.

    method : str
        The type of fingerprint to use. Default is MACCS keys.

    n_bits : int
        The length of the fingerprint.

    Returns
    -------
    array
        The fingerprint array.

    """

    if method == "maccs":
        return list(MACCSkeys.GenMACCSKeys(mol))
    elif method == "ecfp4":
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits))
    elif method == "ecfp6":
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits))
    else:
        print(f"Warning: Wrong method specified: {method}. Default will be used instead.")
        return list(MACCSkeys.GenMACCSKeys(mol))

#### QSAR Modeling

Machine learning (ML)
ML text is adapted from the `sckit-learn` documentation [here](https://scikit-learn.org/stable/).  More information about the algorithms can be found on thier respective documentation pages. 

Classification: Identify which category an object belongs to (e.g. Active/Inactive bioassay respones, Toxic/non-toxic in humans)  
Regression: Prediction of a continuous-values attribute associated with an object (e.g., $\mu$m, AC$_50$ respones in bioassays)

A learning algorithm creates rules by finding patterns in the training data.

* Random Forest (RF): Ensemble of decision trees. A single decision tree splits the features of the input vector in a way that maximizes an objective function. In the random forest algorithm, the trees that are grown are de-correlated because the choice of features for the splits are chosen randomly.
* k-Nearest Neighbors (kNNs): A kNN predicts from a simple majority vote of the nearest neighbors of each point: a query point is assigned the data class which has the most representatives within the nearest neighbors found in the training set.

Most algorithms can be used as both classiers and regressors, however, there are certain algorithms that are not.  The two mentioned above that we will use in this lab are applicable for both.  Scikit-learn has seperate modules for classification and regression.  Therefore, if we want to use RF for classification, we have to import it a seperate object from RF for regression. 

To simulate "real-world" scenarios and to ensures your models are not overfit it is best practice to seperate the compounds being used for QSAR into __training__, and __test__ sets.  Training sets are used for tuning the model parameters and finding patterns usually through cross validation, which we will go over next lab and will ignore this time.  The test set is then used to evaluate the model's accuracy at predicting "new" or "unseen" data. __Test Set Data is the MOST important model validation data__

## Clasification

#### Data Preperation

First we split the data into training and set sets (a portain of the training set will be used as a validation set later).  Typically the QSAR table will consists of an `X` matrix consisting of predictor variables, which in our cases are molecular descriptors and is of the shape (N, M), where N is the number of compounds and M is the number of descriptors.  The second matix `y`,  consits of the values to be predicted, e.g., the classifications of bioassay calls or AC$_{50}$ values, in the case of classification and regression, respectively.  

First we prepare these two matrices.  

It is usually good practice to summarize/check/visualize the data in which you are trying to learn. 

#### Classification

The `train_test_split` function from `scikitlearn` splits our data to a provide test set size.  Here, we'll leave out 20% of our compounds as a test set.  

Get the algorithms from `scikit-learn`.

### Train the models

Use the training data to fit the models

#### Regression

For regression, we can use the same approach, but we need to change our `y` training vector to be the true continious values. 

Most of the algorithms in `sckit-learn` have both a `Classifer` and a `Regressor` interface.  Here we'll import the `Regressors`.

It is usually good practice to visualize the data in which you are trying to learn. 

The metrics used to evaluate classication models are not the same as those used for evaluating regression models.  Typically, there are two metrics __r$^2$__ and the __mean absolute error (MAE)__.